In [30]:
import os

In [29]:
import json
import pandas as pd
import traceback
from langchain_groq import ChatGroq
import os
import langchain
import langchain_groq

from dotenv import load_dotenv

load_dotenv(dotenv_path="src/.env")

key = os.getenv("GROQ_API_KEY")
print("GROQ_API_KEY:",GROQ_API_KEY )



NameError: name 'GROQ_API_KEY' is not defined

In [2]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain



In [16]:
from langchain.callbacks.base import BaseCallbackHandler

class GroqCallbackHandler(BaseCallbackHandler):
    def __init__(self):
        self.request_count = 0

    def on_llm_start(self, serialized, prompts, **kwargs):
        self.request_count += 1
        print(f"Request #{self.request_count} started with prompts: {prompts}")

    def on_llm_end(self, response, **kwargs):
        print(f"Request #{self.request_count} finished with response: {response}\n")

    def on_llm_error(self, error, **kwargs):
        print(f"Request #{self.request_count} encountered an error: {error}\n")



In [15]:
llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="gpt-3.5-turbo", temperature=0.5)

NameError: name 'GROQ_API_KEY' is not defined

In [9]:
from  langchain.callbacks import  GroqCallbackHandler


ImportError: cannot import name 'GroqCallbackHandler' from 'langchain.callbacks' (c:\Users\Sai Sanjith\mcqgenerator\venv\Lib\site-packages\langchain\callbacks\__init__.py)